In [1]:
import numpy as np 
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
#Setting index for easier usage
train = pd.read_csv("train.csv").set_index("PassengerId")
test = pd.read_csv("test.csv").set_index("PassengerId")

train = train.drop("Cabin", axis = 1)
test = test.drop("Cabin", axis = 1)


#Dropping cabin column as a whole and also removing rows that are missing an Age value
# train = train[train.Age.notnull()].drop("Cabin", axis = 1)
# test = test[test.Age.notnull()].drop("Cabin", axis = 1)

#Filling the two missing values in Embarked that I found on google
train.Embarked = train.Embarked.fillna("S")

#Removing ticket prefix,replacing the special value of ticket and then converting all to floats
train = train[train.Ticket != "LINE"]
test = test[test.Ticket != "LINE"]
train.Ticket = train.Ticket.map(lambda t : t.split()[-1]).astype("float")
test.Ticket = test.Ticket.map(lambda t : t.split()[-1]).astype("float")

#Replacing each value of Embarked with a respective integer
# train.Embarked = train.Embarked.replace("S", 1).replace("Q", 2).replace("C", 3)
# test.Embarked = test.Embarked.replace("S", 1).replace("Q", 2).replace("C", 3)

#Same with sex
train.Sex = train.Sex.replace("male", 1).replace("female", 0)
test.Sex = test.Sex.replace("male", 1).replace("female", 0)

dummies = pd.get_dummies(train.Pclass, prefix = "class")
dummies_embark = pd.get_dummies(train.Embarked, prefix = "loc")
train = pd.concat([train, dummies, dummies_embark], axis = 1).drop("Pclass", axis = 1).drop("Embarked", axis = 1)

dummies = pd.get_dummies(test.Pclass, prefix = "class")
dummies_embark = pd.get_dummies(test.Embarked, prefix = "loc")
test = pd.concat([test, dummies, dummies_embark], axis = 1).drop("Pclass", axis = 1).drop("Embarked", axis = 1)

#Replacing titles with dummy
def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'

def replace_titles(x):
    title = x['Title']
    if title in ['Capt', 'Col', 'Don', 'Jonkheer', 'Major', 'Rev', 'Sir']:
        return 'Mr'
    elif title in ['the Countess', 'Mme', 'Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
    
train['Title'] = train['Name'].map(lambda x: get_title(x))
train['Title'] = train.apply(replace_titles, axis=1)

test['Title'] = test['Name'].map(lambda x: get_title(x))
test['Title'] = test.apply(replace_titles, axis=1)

dummies = pd.get_dummies(train.Title)
test_dummies = pd.get_dummies(test.Title)

train = pd.concat([train, dummies], axis = 1).drop("Name", axis = 1).drop("Title", axis = 1)
test = pd.concat([test, test_dummies], axis =1).drop("Name", axis = 1).drop("Title", axis = 1)

full = pd.concat([train, test])

scaler = StandardScaler()

ticket = full.Ticket.to_numpy().reshape(-1, 1)
fare = full.Fare.to_numpy().reshape(-1, 1)
age = full.Age.to_numpy().reshape(-1, 1)

full.Ticket = scaler.fit_transform(ticket)
full.Fare = scaler.fit_transform(fare)
full.Age = scaler.fit_transform(age)

#Dealing with missing ages

imputer = IterativeImputer()
features = ["Age", "Pclass"]

full = pd.DataFrame(imputer.fit_transform(full), index = full.index, columns = full.columns)

#Iterative imputation of train Age category
train = full[:len(train)]
test = full[len(train):]


In [3]:
features = ["Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "class_1", "class_2", "class_3", "loc_C", "loc_Q", "loc_S", "Master", "Miss", "Mr", "Mrs"]

#Converting the features to numpy arrays
X = train[features].to_numpy()
X_test = test[features].to_numpy().transpose()

X_test = X_test.T

#Same with Y
Y = train["Survived"].to_numpy()
Y = Y.reshape((Y.shape[0], 1))


#Splitting into 80-20 train dev split
#TODO: MANUALLY SPLIT, MAKE THE AVG AGE DISTRIBUTION BE IN 
X, X_dev, Y, Y_dev = train_test_split(X, Y, test_size = 0.25)


#Transposing all so they have correct dimentions
X = X.astype("float32")
X_dev = X_dev.astype("float32")
Y = Y.astype("float32")
Y_dev = Y_dev.astype("float32")


In [25]:
model = tf.keras.Sequential()

init_he = tf.keras.initializers.HeNormal()
init_x = tf.keras.initializers.GlorotNormal()

model.add(tf.keras.layers.Dense(128, activation = "relu", kernel_initializer = init_he))
model.add(tf.keras.layers.Dropout(rate = 0.25))
model.add(tf.keras.layers.Dense(1, activation = "sigmoid", kernel_initializer = init_x))

cost = tf.keras.losses.BinaryCrossentropy(from_logits = True)
opt = tf.keras.optimizers.RMSprop(learning_rate = 0.015)


model.compile(optimizer = opt, loss = cost, metrics = ["accuracy"])

model.fit(X, Y, batch_size = 8, epochs = 50)



Epoch 1/50
111/111 [==============================] - 0s 592us/step - loss: 0.6430 - accuracy: 0.7756
Epoch 2/50
111/111 [==============================] - 0s 512us/step - loss: 0.6246 - accuracy: 0.8230
Epoch 3/50
111/111 [==============================] - 0s 575us/step - loss: 0.6199 - accuracy: 0.8230
Epoch 4/50
111/111 [==============================] - 0s 566us/step - loss: 0.6169 - accuracy: 0.8343
Epoch 5/50
111/111 [==============================] - 0s 584us/step - loss: 0.6129 - accuracy: 0.8467
Epoch 6/50
111/111 [==============================] - 0s 629us/step - loss: 0.6118 - accuracy: 0.8444
Epoch 7/50
111/111 [==============================] - 0s 593us/step - loss: 0.6109 - accuracy: 0.8478
Epoch 8/50
111/111 [==============================] - 0s 575us/step - loss: 0.6092 - accuracy: 0.8489
Epoch 9/50
111/111 [==============================] - 0s 657us/step - loss: 0.6092 - accuracy: 0.8501
Epoch 10/50
111/111 [==============================] - 0s 525us/step - loss: 0.613

In [6]:
model.evaluate(X, Y)
#model.evaluate(X_dev, Y_dev)

28/28 [==============================] - 0s 512us/step - loss: 0.6001 - accuracy: 0.8703


[0.6000521183013916, 0.8703494668006897]

In [26]:
def write_to_csv(filename, predictions, passID):

    new_df = pd.DataFrame({"PassengerID": passID, "Survived":predictions}, index = [0] * len(passID))
    
    new_df.to_csv(filename + ".csv", index = False)
    
pred = model.predict(X_test).round().astype(int).reshape(-1,)

passID = test.index.to_numpy().astype(int)

write_to_csv("dropout0.25_50iter", pred, passID)

In [41]:
test.index

Int64Index([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,
            ...
            1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309],
           dtype='int64', name='PassengerId', length=418)